In [70]:
import asyncio
import os
import re
from contextlib import closing

import aiohttp
import requests
from bs4 import BeautifulSoup

In [66]:
resp = requests.get('http://goalkicker.com/')
soup = BeautifulSoup(resp.content, 'html.parser')

In [67]:
urls = []

for link in soup.find_all('a'):
    urls.append('http://goalkicker.com/{}'.format(link.get('href')))

In [75]:
re_onclick = re.compile(r"^location.href='(?P<name>\w+\.\w+)'$")

pdfs = {}

for url in urls:
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, 'html.parser')

    btn_dl = soup.find('button', attrs={'class': 'download'})

    pdf_filename = re_onclick.match(btn_dl.get('onclick')).groups()[0]
    pdfs[pdf_filename] = '{0}/{1}'.format(url, pdf_filename)

In [74]:
@asyncio.coroutine
def download(url, filename, session, semaphore, chunk_size=1<<15):
    with (yield from semaphore):
        print('Downloading {}'.format(filename))
    
        resp = yield from session.get(url)
    
        with closing(resp), open(os.path.join('pdf/', filename), 'wb') as fobj:
            while True:
                chunk = yield from resp.content.read(chunk_size)
            
                if not chunk:
                    break
            
                fobj.write(chunk)
    
            print('Done {}'.format(filename))
        
        return filename, (response.status, tuple(response.headers.items()))

with closing(asyncio.get_event_loop()) as loop, \
        closing(aiohttp.ClientSession()) as session:

    semaphore = asyncio.Semaphore(4)
    download_tasks = (download(url, filename, session, semaphore) for filename, url in pdfs.items())
    result = loop.run_until_complete(asyncio.gather(*download_tasks))

Creating a client session outside of coroutine
client_session: <aiohttp.client.ClientSession object at 0x104cb3668>


RuntimeError: Event loop is closed